In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [61]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data = data.iloc[start_index+3:end_index-1,:]
    data.columns = columns
    if('History' in filename):
        drop_columns = ['Transacción', 'Swap', 'Orden', 'Comentario', 'Tasa']
    else:
        drop_columns = ['Transacción', 'Swap', 'Orden', 'Comentario']
    data = data.drop(drop_columns, axis=1)
    data = data.set_index('Fecha/Hora')
    data.index.name = 'Fecha'
    return data

In [ ]:
filename='ReportTester-4000009439.xlsx'
df = load_xls_report(filename)
df.head()

In [99]:
def expected_profitability(df):
    profit_trades = df[(df['Dirección']=='out') & (df['Beneficio']>0)]['Beneficio'].count()
    profit_avg = df[(df['Dirección']=='out') & (df['Beneficio']>0)]['Beneficio'].mean()
    loss_trades = df[(df['Dirección']=='out') & (df['Beneficio']<0)]['Beneficio'].count()
    loss_avg = df[(df['Dirección']=='out') & (df['Beneficio']<0)]['Beneficio'].mean()
    beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
    total_trades = profit_trades + loss_trades
    expected_profit = beneficio_neto / total_trades
    return expected_profit

In [100]:

print(f"Rentabilidad esperada: {expected_profitability(df):.2f}")

Rentabilidad esperada: -0.03
